In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import cv2
from skimage.io import imread
import keras
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
from tensorflow.keras.applications import VGG16, InceptionResNetV2
from keras import regularizers
from tensorflow.keras.optimizers import Adam

In [11]:
train_set = pd.read_csv('train.csv', index_col=0)
train_set

,id_img,paths,label
0,100,data\train\happy\100.jpg,happy
1,10000,data\train\happy\10000.jpg,happy
2,10001,data\train\happy\10001.jpg,happy
3,10005,data\train\happy\10005.jpg,happy
4,10035,data\train\happy\10035.jpg,happy
...,...,...,...
12097,9966,data\train\sadness\9966.jpg,sadness
12098,9974,data\train\sadness\9974.jpg,sadness
12099,9976,data\train\sadness\9976.jpg,sadness
12100,9986,data\train\sadness\9986.jpg,sadness


In [12]:
for i in train_set['paths'].index:
    train_set['paths'].loc[i] = train_set['paths'][i].replace("\\", "/")
print(train_set)

/var/folders/pm/rkf6f1wx30z1g4q9vzk8z4540000gn/T/ipykernel_26757/23886526.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['paths'].loc[i] = train_set['paths'][i].replace("\\", "/")


       id_img                        paths    label
0         100     data/train/happy/100.jpg    happy
1       10000   data/train/happy/10000.jpg    happy
2       10001   data/train/happy/10001.jpg    happy
3       10005   data/train/happy/10005.jpg    happy
4       10035   data/train/happy/10035.jpg    happy
...       ...                          ...      ...
12097    9966  data/train/sadness/9966.jpg  sadness
12098    9974  data/train/sadness/9974.jpg  sadness
12099    9976  data/train/sadness/9976.jpg  sadness
12100    9986  data/train/sadness/9986.jpg  sadness
12101    9997  data/train/sadness/9997.jpg  sadness

[12102 rows x 3 columns]


In [6]:
train_set['array'] = train_set['paths'].apply(lambda x: cv2.imread("data/train/" + x))

[ WARN:0@40.387] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('data/train/data/train/happy/100.jpg'): can't open/read file: check file path/integrity
[ WARN:0@40.387] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('data/train/data/train/happy/10000.jpg'): can't open/read file: check file path/integrity
[ WARN:0@40.387] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('data/train/data/train/happy/10001.jpg'): can't open/read file: check file path/integrity
[ WARN:0@40.387] global /Users/xperience/actions-runner/_work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp (239) findDecoder imread_('data/train/data/train/happy/10005.jpg'): can't open/read file: check file path/integrity
[ WARN:0@4

In [13]:
lista = []
for i in train_set["paths"]:
    lista.append(imread(i, 0))

In [14]:
x = np.array(lista)

In [15]:
values = {"happy":0, "sadness":1}

In [16]:
y = train_set["label"].replace(values)

In [17]:
print(x.shape)
print(y.shape)

(12102, 48, 48)
(12102,)


In [19]:
from sklearn.utils import shuffle

x = shuffle(x, random_state = 30)
y = shuffle(y, random_state = 30)

In [25]:
validation_datagen = ImageDataGenerator(rescale = 1./255,
                                         validation_split = 0.2)

In [26]:
train_datagen = ImageDataGenerator(rotation_range = 180,
                                         width_shift_range = 0.1,
                                         height_shift_range = 0.1,
                                         horizontal_flip = True,
                                         rescale = 1./255,
                                         zoom_range = 0.2,
                                         validation_split = 0.2
                                        )

In [27]:
model= tf.keras.models.Sequential()

In [28]:

model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense (128, activation='relu')) 
model.add(Dropout(0.25))
model.add(Dense (64, activation='relu'))
model.add(Dense(7, activation= 'softmax'))

In [29]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

earlystop = EarlyStopping(patience=10)
mcheckpoint = ModelCheckpoint('callback_model.h5')

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 46, 46, 64)        640       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 23, 23, 64)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 23, 23, 64)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 10, 10, 128)      0         
 2D)                                                             
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 128)      

In [33]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [34]:
istory = model.fit(x,y, epochs =50, batch_size=128, callbacks= [earlystop, mcheckpoint], validation_split=0.1)

Epoch 1/50


2022-10-07 22:17:04.006519: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - ETA: 0s - loss: 0.2670 - accuracy: 0.8840

2022-10-07 22:17:09.806942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


86/86 [==============================] - 6s 68ms/step - loss: 0.2670 - accuracy: 0.8840 - val_loss: 0.5226 - val_accuracy: 0.8076
Epoch 2/50
86/86 [==============================] - 6s 66ms/step - loss: 0.2524 - accuracy: 0.8922 - val_loss: 0.5229 - val_accuracy: 0.8274
Epoch 3/50
86/86 [==============================] - 6s 66ms/step - loss: 0.2479 - accuracy: 0.8942 - val_loss: 0.4758 - val_accuracy: 0.8315
Epoch 4/50
86/86 [==============================] - 6s 66ms/step - loss: 0.2441 - accuracy: 0.9018 - val_loss: 0.4543 - val_accuracy: 0.8200
Epoch 5/50
86/86 [==============================] - 6s 67ms/step - loss: 0.2172 - accuracy: 0.9116 - val_loss: 0.5629 - val_accuracy: 0.8233
Epoch 6/50
86/86 [==============================] - 6s 66ms/step - loss: 0.2115 - accuracy: 0.9123 - val_loss: 0.4620 - val_accuracy: 0.8357
Epoch 7/50
86/86 [==============================] - 6s 65ms/step - loss: 0.2048 - accuracy: 0.9144 - val_loss: 0.5516 - val_accuracy: 0.8365
Epoch 8/50
86/86 [======